In [17]:
#%matplotlib widget
import sympy as sp
import numpy as np
import matplotlib.pylab as plt

In [18]:
E_f, eps_cu, eps_cy, sig_cu, sig_cy, f_fu, f_cm = \
 sp.symbols(r'E_f, epsilon_cu, epsilon_cy, sigma_cu, sigma_cy, f_fu, f_cm', positive=True)
rho_f, delta_eps = \
 sp.symbols(r'rho_f, epsilon_Delta', positive=True)

$$
 \varepsilon_\mathrm{cu}= 10^{-3} \cdot
  \begin{cases}
      3.5,
    & f_\mathrm{cm} \le 58~\mathrm{MPa} \\
    2.8 + 27 \cdot \big(\frac{98 - f_\mathrm{cm}}{100}\big)^4,
    & 58~\mathrm{MPa} < f_\mathrm{cm} \le 98~\mathrm{MPa} \\
    2.8,
    & {f_\mathrm{cm} > 98~\mathrm{MPa}}.
  \end{cases}
$$

In [19]:
eps_cu_ = sp.Rational(1, 1000) * sp.Piecewise(
    (3.5, f_cm <= 58),
    (2.8 + 27 * ((98-f_cm)/100)**4, f_cm <= 98),
    (2.8, True)
)
eps_cu_

Piecewise((3.5, f_cm <= 58), (27*(49/50 - f_cm/100)**4 + 2.8, f_cm <= 98), (2.8, True))/1000

$$
\boxed{
\rho_\mathrm{fb} = \frac{E_\mathrm{f} \left(\sigma_\mathrm{cu} \varepsilon_\mathrm{cu}~-~\sigma_\mathrm{cu} \varepsilon_\mathrm{cy}~+~\sigma_\mathrm{cy} \varepsilon_\mathrm{cu}\right)}{2 f_\mathrm{fu} \left(E_\mathrm{f} \varepsilon_\mathrm{cu}~+~f_\mathrm{fu}\right)}
,}
$$

In [33]:
rho_fb_ = ((E_f * (sig_cu * eps_cu - sig_cu * eps_cy + sig_cy * eps_cu))
          /(2 * f_fu * (E_f * eps_cu + f_fu)))
rho_fb_

E_f*(epsilon_cu*sigma_cu + epsilon_cu*sigma_cy - epsilon_cy*sigma_cu)/(2*f_fu*(E_f*epsilon_cu + f_fu))

$$
\sigma_\mathrm{f} =
\frac{1}{2} \left( \sqrt{\frac{E_\mathrm{f} \cdot \big(E_\mathrm{f} \rho_\mathrm{f} \varepsilon_\mathrm{cu}^2~+~2 \sigma_\mathrm{cu} \Delta\varepsilon~+~2 \sigma_\mathrm{cy} \varepsilon_\mathrm{cu} \big)}
{\rho_\mathrm{f}}}
- E_{f} \varepsilon_\mathrm{cu}\right) \le f_\mathrm{fu}.

$$

In [21]:
sig_f = sp.Rational(1,2) * (
    sp.sqrt((E_f * (E_f*rho_f*eps_cu**2 + 2*sig_cu*delta_eps + 2 * sig_cy*eps_cu))/
            (rho_f)
    ) - E_f * eps_cu
)
sig_f

sqrt(E_f)*sqrt(E_f*epsilon_cu**2*rho_f + 2*epsilon_Delta*sigma_cu + 2*epsilon_cu*sigma_cy)/(2*sqrt(rho_f)) - E_f*epsilon_cu/2

$$
c_\mathrm{e, total} = \frac{A_\mathrm{tri} c_\mathrm{e, tri} + A_\mathrm{trab} c_\mathrm{e, trab}}{A_\mathrm{tri} + A_\mathrm{trab}}
$$

In [22]:
b, d, h, z = sp.symbols('b, d, h, z', positive=True)
m_0 = sp.symbols('m_0')
h_ = d + z
A_f_ = rho_f * b * d

$$
M_\mathrm{u,cu} = \frac{A_\mathrm{f} d \sigma_\mathrm{f}}{3 m_0 \left(E_\mathrm{f} \varepsilon_\mathrm{cu} + \sigma_\mathrm{f}\right)}
\bigg(
3 m_0 \left(E_\mathrm{f} \varepsilon_\mathrm{cu} + \sigma_\mathrm{f}\right)
- ~E_\mathrm{f} \left(\varepsilon_\Delta^2 \left(2 \sigma_\mathrm{cy} + \sigma_\mathrm{cu} \right)
~+~\sigma_\mathrm{cy} \varepsilon_\mathrm{cy} \left(3 \varepsilon_\mathrm{cu} - 2 \varepsilon_\mathrm{cy}\right)\right)\bigg)
$$

In [37]:
M_u_cu_ = ((A_f_ * d * sig_f) / (3*m_0*(E_f * eps_cu + sig_f)) *
           (3*m_0*(E_f*eps_cu+sig_f) - E_f*(delta_eps**2*(2*sig_cy+sig_cu)) + 
           sig_cy * eps_cy * (3*eps_cu-2*eps_cy))
          )
sp.simplify(M_u_cu_)

b*d**2*sqrt(rho_f)*(sqrt(E_f)*sqrt(E_f*epsilon_cu**2*rho_f + 2*epsilon_Delta*sigma_cu + 2*epsilon_cu*sigma_cy) - E_f*epsilon_cu*sqrt(rho_f))*(3*m_0*(sqrt(E_f)*sqrt(E_f*epsilon_cu**2*rho_f + 2*epsilon_Delta*sigma_cu + 2*epsilon_cu*sigma_cy) + E_f*epsilon_cu*sqrt(rho_f)) + 2*sqrt(rho_f)*(-E_f*epsilon_Delta**2*(sigma_cu + 2*sigma_cy) + epsilon_cy*sigma_cy*(3*epsilon_cu - 2*epsilon_cy)))/(6*m_0*(sqrt(E_f)*sqrt(E_f*epsilon_cu**2*rho_f + 2*epsilon_Delta*sigma_cu + 2*epsilon_cu*sigma_cy) + E_f*epsilon_cu*sqrt(rho_f)))

In [24]:
m_0_ = delta_eps * sig_cu  + delta_eps * sig_cy + eps_cy * sig_cy

In [25]:
M_u_cu__ = sp.simplify(M_u_cu_.subs(m_0, m_0_))

In [26]:
M_u_cu___ = sp.simplify(M_u_cu__.subs(rho_f, rho_fb_))
M_u_cu___

E_f*b*d**2*(2*(E_f*epsilon_Delta**2*(sigma_cu + 2*sigma_cy) - epsilon_cy*sigma_cy*(3*epsilon_cu - 2*epsilon_cy))*sqrt(epsilon_cu*sigma_cu + epsilon_cu*sigma_cy - epsilon_cy*sigma_cu) - 3*(E_f*epsilon_cu*sqrt(epsilon_cu*sigma_cu + epsilon_cu*sigma_cy - epsilon_cy*sigma_cu) + sqrt(E_f**2*epsilon_cu**2*(epsilon_cu*sigma_cu + epsilon_cu*sigma_cy - epsilon_cy*sigma_cu) + 4*f_fu*(E_f*epsilon_cu + f_fu)*(epsilon_Delta*sigma_cu + epsilon_cu*sigma_cy)))*(epsilon_Delta*sigma_cu + epsilon_Delta*sigma_cy + epsilon_cy*sigma_cy))*(E_f*epsilon_cu*sqrt(epsilon_cu*sigma_cu + epsilon_cu*sigma_cy - epsilon_cy*sigma_cu) - sqrt(E_f**2*epsilon_cu**2*(epsilon_cu*sigma_cu + epsilon_cu*sigma_cy - epsilon_cy*sigma_cu) + 4*f_fu*(E_f*epsilon_cu + f_fu)*(epsilon_Delta*sigma_cu + epsilon_cu*sigma_cy)))*sqrt(epsilon_cu*sigma_cu + epsilon_cu*sigma_cy - epsilon_cy*sigma_cu)/(12*f_fu*(E_f*epsilon_cu + f_fu)*(E_f*epsilon_cu*sqrt(epsilon_cu*sigma_cu + epsilon_cu*sigma_cy - epsilon_cy*sigma_cu) + sqrt(E_f**2*epsilon_cu**2

In [27]:
19 * 1.8

34.2

In [28]:
1.8 / 8.4

0.21428571428571427

In [29]:
M_E = sp.symbols('M_E', positive=True)

In [30]:
sp.solve(M_u_cu___ - M_E, d)[0]

-sqrt(6)*sqrt(M_E)*sqrt(f_fu)*sqrt((E_f*epsilon_cu*sqrt(epsilon_cu*sigma_cu + epsilon_cu*sigma_cy - epsilon_cy*sigma_cu) + sqrt(E_f**2*epsilon_cu**3*sigma_cu + E_f**2*epsilon_cu**3*sigma_cy - E_f**2*epsilon_cu**2*epsilon_cy*sigma_cu + 4*E_f*epsilon_Delta*epsilon_cu*f_fu*sigma_cu + 4*E_f*epsilon_cu**2*f_fu*sigma_cy + 4*epsilon_Delta*f_fu**2*sigma_cu + 4*epsilon_cu*f_fu**2*sigma_cy))/(E_f**2*epsilon_Delta**2*epsilon_cu**2*sigma_cu**2*sqrt(epsilon_cu*sigma_cu + epsilon_cu*sigma_cy - epsilon_cy*sigma_cu) + 3*E_f**2*epsilon_Delta**2*epsilon_cu**2*sigma_cu*sigma_cy*sqrt(epsilon_cu*sigma_cu + epsilon_cu*sigma_cy - epsilon_cy*sigma_cu) + 2*E_f**2*epsilon_Delta**2*epsilon_cu**2*sigma_cy**2*sqrt(epsilon_cu*sigma_cu + epsilon_cu*sigma_cy - epsilon_cy*sigma_cu) - E_f**2*epsilon_Delta**2*epsilon_cu*epsilon_cy*sigma_cu**2*sqrt(epsilon_cu*sigma_cu + epsilon_cu*sigma_cy - epsilon_cy*sigma_cu) - 2*E_f**2*epsilon_Delta**2*epsilon_cu*epsilon_cy*sigma_cu*sigma_cy*sqrt(epsilon_cu*sigma_cu + epsilon_cu*sigm

In [31]:
p_0 = E_f * eps_cu
p_1 = 3 * eps_cu * f_fu
p_2 = sig_cu * eps_cy
p_3 = E_f * eps_cu**2
p_4 = (- E_f * sig_cu * eps_cy**2 + eps_cu * p_1 + 2 * sig_cu * p_3 + sig_cy * eps_cy * p_0 +
       sig_cy * p_1 + sig_cy * p_3 - p_0 * p_2 - 3 * f_fu * p_2
      )

In [39]:
sp.simplify(p_4)

2*E_f*epsilon_cu**2*sigma_cu + E_f*epsilon_cu**2*sigma_cy - E_f*epsilon_cu*epsilon_cy*sigma_cu + E_f*epsilon_cu*epsilon_cy*sigma_cy - E_f*epsilon_cy**2*sigma_cu + 3*epsilon_cu**2*f_fu + 3*epsilon_cu*f_fu*sigma_cy - 3*epsilon_cy*f_fu*sigma_cu